In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset

In [2]:
from google.colab import files
uploaded = files.upload()

Saving glaucoma_data.csv to glaucoma_data.csv


In [3]:
df = pd.read_csv(next(iter(uploaded.keys())))
print(df.head())

                                 question  \
0                What is (are) Glaucoma ?   
1                  What causes Glaucoma ?   
2     What are the symptoms of Glaucoma ?   
3  What are the treatments for Glaucoma ?   
4                What is (are) Glaucoma ?   

                                              answer           source  \
0  Glaucoma is a group of diseases that can damag...  NIHSeniorHealth   
1  Nearly 2.7 million people have glaucoma, a lea...  NIHSeniorHealth   
2  Symptoms of Glaucoma  Glaucoma can develop in ...  NIHSeniorHealth   
3  Although open-angle glaucoma cannot be cured, ...  NIHSeniorHealth   
4  Glaucoma is a group of diseases that can damag...  NIHSeniorHealth   

  focus_area  
0   Glaucoma  
1   Glaucoma  
2   Glaucoma  
3   Glaucoma  
4   Glaucoma  


In [4]:
class QADataset(Dataset):
    def _init_(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_len = max_len

    def _len_(self):
        return len(self.data)

    def _getitem_(self, index):
        question = str(self.data.iloc[index, 0])
        answer = str(self.data.iloc[index, 1])

        encoding = self.tokenizer(
            question,
            answer,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        # Get the token positions of the answer in the input sequence
        inputs = self.tokenizer(question, answer, add_special_tokens=True, return_offsets_mapping=True)
        offset_mapping = inputs.pop("offset_mapping")

        # Find the start and end positions of the answer in the input tokens
        # Assuming answer is a substring of the context. If not adjust logic
        start_char = question.find(answer)  # Find the start position of the answer within the question
        end_char = start_char + len(answer)

        # Initialize start and end token positions
        start_token_index = -1
        end_token_index = -1

        # Iterate over offset mapping to find corresponding token indices
        # Check if offset_mapping[0] has sub-elements before iterating
        if offset_mapping and offset_mapping[0]:
            for i, offset in enumerate(offset_mapping[0]):  # Changed to iterate over 'offset'
                # Check if offset is iterable before unpacking
                if isinstance(offset, (tuple, list)) and len(offset) == 2:
                    start, end = offset
                    if start <= start_char and start_char <= end:
                        start_token_index = i
                    if start <= end_char and end_char <= end:
                        end_token_index = i
                        break  # Exit loop once end token is found
                #else:
                #    print(f"Warning: Unexpected offset format at index {i}: {offset}")  # Optional: Print a warning for unexpected offset formats
        else:
            # Handle the case where offset_mapping[0] is empty or does not have sub-elements
            start_token_index = 0  # or another appropriate default value
            end_token_index = 0    # or another appropriate default value

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        # Ensure start_positions and end_positions are within the sequence length
        start_token_index = min(start_token_index, len(input_ids) - 1)
        end_token_index = min(end_token_index, len(input_ids) - 1)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor(start_token_index, dtype=torch.long),  # Provide start position
            'end_positions': torch.tensor(end_token_index, dtype=torch.long),
        }

In [7]:
class QADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len): # Changed _init_ to __init__
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_len = max_len

    def __len__(self): # Changed _len_ to __len__
        return len(self.data)

    def __getitem__(self, index): # Changed _getitem_ to __getitem__
        question = str(self.data.iloc[index, 0])
        answer = str(self.data.iloc[index, 1])

        encoding = self.tokenizer(
            question,
            answer,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        # Get the token positions of the answer in the input sequence
        inputs = self.tokenizer(question, answer, add_special_tokens=True, return_offsets_mapping=True)
        offset_mapping = inputs.pop("offset_mapping")

        # Find the start and end positions of the answer in the input tokens
        # Assuming answer is a substring of the context. If not adjust logic
        start_char = question.find(answer)  # Find the start position of the answer within the question
        end_char = start_char + len(answer)

        # Initialize start and end token positions
        start_token_index = -1
        end_token_index = -1

        # Iterate over offset mapping to find corresponding token indices
        # Check if offset_mapping[0] has sub-elements before iterating
        if offset_mapping and offset_mapping[0]:
            for i, offset in enumerate(offset_mapping[0]):  # Changed to iterate over 'offset'
                # Check if offset is iterable before unpacking
                if isinstance(offset, (tuple, list)) and len(offset) == 2:
                    start, end = offset
                    if start <= start_char and start_char <= end:
                        start_token_index = i
                    if start <= end_char and end_char <= end:
                        end_token_index = i
                        break  # Exit loop once end token is found
                #else:
                #    print(f"Warning: Unexpected offset format at index {i}: {offset}")  # Optional: Print a warning for unexpected offset formats
        else:
            # Handle the case where offset_mapping[0] is empty or does not have sub-elements
            start_token_index = 0  # or another appropriate default value
            end_token_index = 0    # or another appropriate default value

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        # Ensure start_positions and end_positions are within the sequence length
        start_token_index = min(start_token_index, len(input_ids) - 1)
        end_token_index = min(end_token_index, len(input_ids) - 1)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor(start_token_index, dtype=torch.long),  # Provide start position
            'end_positions': torch.tensor(end_token_index, dtype=torch.long),
        }

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
max_len = 256
train_dataset = QADataset(df, tokenizer, max_len)

In [9]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Step 6: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Token indices sequence length is longer than the specified maximum sequence length for this model (727 > 512). Running this sequence through the model will result in indexing errors


Step,Training Loss
10,5.620600
20,5.490300
30,5.207400
40,4.731000
50,4.096800
60,3.347400
70,2.546400
80,1.657700
90,0.853000
100,0.315100


Step,Training Loss
10,5.620600
20,5.490300
30,5.207400
40,4.731000
50,4.096800
60,3.347400
70,2.546400
80,1.657700
90,0.853000
100,0.315100
